In [2]:
from fastai.vision.all import *
from models.swin_transformer_v2 import SwinTransformerV2
from fastai.layers import *
import torch
import models
from torchvision import transforms
from torchvision.transforms import InterpolationMode
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
import PIL

/home/ubuntu/miniconda3/envs/fastai/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Warning] Fused window process have not been installed. Please refer to get_started.md for installation.
Tutel has not been installed. To use Swin-MoE, please install Tutel; otherwise, just ignore this.


In [3]:
defaults.device = torch.device('cuda')

In [4]:
def label_func(x):
    return x.parent.name

In [5]:
class SiameseImage(fastuple):
    def show(self, ctx=None, **kwargs): 
        if len(self) > 2:
            img1,img2,similarity = self
        else:
            img1,img2 = self
            similarity = 'Undetermined'
        if not isinstance(img1, Tensor):
            if img2.size != img1.size: img2 = img2.resize(img1.size)
            t1,t2 = tensor(img1),tensor(img2)
            t1,t2 = t1.permute(2,0,1),t2.permute(2,0,1)
        else: t1,t2 = img1,img2
        line = t1.new_zeros(t1.shape[0], t1.shape[1], 10)
        return show_image(torch.cat([t1,line,t2], dim=2), title=similarity, ctx=ctx, **kwargs)

In [6]:
class SiameseTransform(Transform):
    def __init__(self, files, splits):
        self.splbl2files = [{l: [f for f in files[splits[i]] if label_func(f) == l] for l in labels}
                          for i in range(2)]
        self.valid = {f: self._draw(f,1) for f in files[splits[1]]}
    def encodes(self, f):
        f2,same = self.valid.get(f, self._draw(f,0))
        img1,img2 = PILImage.create(f),PILImage.create(f2)
        return SiameseImage(img1, img2, int(same))
    
    def _draw(self, f, split=0):
        same = random.random() < 0.5
        cls = label_func(f)
        prod = f.name.split("_")[0]
        if not same: cls = random.choice(L(l for l in labels if l != cls)) 
        prod_constrained_files = [file for file in self.splbl2files[split][cls] if file.name.split("_")[0] == prod]
        return random.choice(prod_constrained_files),same

In [7]:
class ImageTuple(fastuple):
    @classmethod
    def create(cls, fns): return cls(tuple(PILImage.create(f) for f in fns))
    
    def show(self, ctx=None, **kwargs): 
        t1,t2 = self
        if not isinstance(t1, Tensor) or not isinstance(t2, Tensor) or t1.shape != t2.shape: return ctx
        line = t1.new_zeros(t1.shape[0], t1.shape[1], 10)
        return show_image(torch.cat([t1,line,t2], dim=2), ctx=ctx, **kwargs)

In [8]:
path = Path("/home/ubuntu/data/mvtec_classification/")

In [9]:
images = get_image_files(path)
labels = list(set(images.map(label_func)))
len(labels)

2

In [10]:
labels

['defective', 'good']

In [35]:
t = []

t.append(transforms.Resize((256, 256), interpolation=InterpolationMode.BICUBIC))
t.append(transforms.ToTensor())
t.append(transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD))
data_transforms = transforms.Compose(t)

In [11]:
splits = GrandparentSplitter(train_name="train", valid_name="test")(images)
tfm = SiameseTransform(images, splits)
tls = TfmdLists(images, tfm, splits=splits)
dls = tls.dataloaders(after_item=[Resize((256, 256)), ToTensor], 
                        batch_size = 16,
                      after_batch=[IntToFloatTensor, Normalize.from_stats(*imagenet_stats)])

In [12]:
swin_model = SwinTransformerV2(
    img_size=256,
    patch_size=4,
    in_chans=3,
    num_classes=1000,
    embed_dim=96,
    depths=[2, 2, 6, 2],
    num_heads=[3, 6, 12, 24],
    window_size=8,
    mlp_ratio=4.0,
    qkv_bias=True,
    drop_rate=0.0,
    drop_path_rate=0.2,
    ape=False,
    patch_norm=True,
    use_checkpoint=True,
    pretrained_window_sizes=[0, 0, 0, 0],
    no_head=True,
)

/home/ubuntu/miniconda3/envs/fastai/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525541990/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [13]:
class SiameseModel(Module):
    def __init__(self, encoder, head):
        self.encoder,self.head = encoder,head
    
    def forward(self, x1, x2):
        ftrs = torch.cat([self.encoder(x1), self.encoder(x2)], dim=1)
        return self.head(ftrs)
    
    #TODO: one image should be hardcoded and other one should come from runtime
    def siampredict(self:Learner, item, rm_type_tfms=None, with_input=False):
        res = self.predict(item, rm_type_tfms=None, with_input=False)
        if res[0] == tensor(0):
            SiameseImage(item[0], item[1], 'Prediction: Not similar').show()
        else:
            SiameseImage(item[0], item[1], 'Prediction: Similar').show()
        return res

In [14]:
encoder = swin_model

In [15]:
class Head(nn.Module):
    def __init__(self, bn_num=3072):
        super(Head, self).__init__()
        self.flatten = nn.Flatten()
        self.bn1 = torch.nn.BatchNorm1d(bn_num, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.dropout1 = torch.nn.Dropout(p=0.5, inplace=False)
        self.linear1 = torch.nn.Linear(in_features=bn_num, out_features=512, bias=True)
        self.relu = torch.nn.ReLU()
        self.bn2 = torch.nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.dropout2 = torch.nn.Dropout(p=0.5, inplace=False)
        self.linear2 = torch.nn.Linear(in_features=512, out_features=2, bias=True)
        

    def forward(self, x):
        x = self.flatten(x)
        x = self.bn1(x)
        x = self.dropout1(x)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.bn2(x)
        x = self.dropout2(x)
        x = self.linear2(x)
        return x

In [16]:
head = Head(bn_num=2000)
model = SiameseModel(encoder, head)

In [17]:
# model = model.to('cuda')

In [18]:
def siamese_splitter(model):
    return [params(model.encoder), params(model.head)]

In [19]:
def loss_func(out, targ):
    return CrossEntropyLossFlat()(out, targ.long())

In [20]:
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), splitter=siamese_splitter, metrics=accuracy)

In [21]:
#load model
learn.load('siamese_swin_tiny')
print(type(learn.model))

<class '__main__.SiameseModel'>


In [ ]:
learn.predict()